# INFO-F-422 -  Statistical Foundations of Machine Learning 

### Student 1 - __[Ghita.Ait.Ouhmane@ulb.ac.be](mailto:Ghita.Ait.Ouhmane@ulb.ac.be) - Student ID 453422__
### Student 2 - __[Ha.Duong@ulb.ac.be](mailto:Ha.Duong@ulb.ac.be) - Student ID 463350__
### Student 3 - __[Maxime.Langlet@ulb.ac.be](mailto:Maxime.Langlet@ulb.ac.be) - Student ID 427821__

### Video presentation: www.youtube.com/abcd1234

## Statistical foundations of Machine Learning - Project 2020-2021


# Introduction


In [1]:
# install the packages and libraries
install.packages("e1071")
install.packages("tidyverse")
install.packages("pillar")
library(rpart)
library(e1071)
library(rpart.plot)##??
library(randomForest)

Installing package into 'C:/Users/duong/OneDrive/Documents/R/win-library/4.0'
(as 'lib' is unspecified)



package 'e1071' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'e1071'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problème lors de la copie de C:\Users\duong\OneDrive\Documents\R\win-library\4.0\00LOCK\e1071\libs\x64\e1071.dll vers C:\Users\duong\OneDrive\Documents\R\win-library\4.0\e1071\libs\x64\e1071.dll: Permission denied"
Warning message:
"restored 'e1071'"



The downloaded binary packages are in
	C:\Users\duong\AppData\Local\Temp\RtmpYZ0r8T\downloaded_packages


Installing package into 'C:/Users/duong/OneDrive/Documents/R/win-library/4.0'
(as 'lib' is unspecified)



package 'tidyverse' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\duong\AppData\Local\Temp\RtmpYZ0r8T\downloaded_packages


Installing package into 'C:/Users/duong/OneDrive/Documents/R/win-library/4.0'
(as 'lib' is unspecified)



package 'pillar' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\duong\AppData\Local\Temp\RtmpYZ0r8T\downloaded_packages


ERROR: Error in library(rpart.plot): there is no package called 'rpart.plot'


'''date_recorded
num_private
funder
installer
wpt_name
subvillage
region? (code 27, region 21, why?)
lga
ward
recorded_by
scheme_name
payment
quantity
waterpoint_type_group'''

In [62]:
# read the datasets

sub_format <- read.csv("database/Pump_it_Up_Data_Mining_the_Water_Table_-_Submission_format.csv", header=TRUE)
set_val_test <- read.csv("database/Pump_it_Up_Data_Mining_the_Water_Table_-_Test_set_values.csv", header=TRUE)
set_labels <- read.csv("database/Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_labels.csv", header=TRUE)
dataset <- read.csv("database/Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_values.csv", header=TRUE)

# print the number of values of the Y
choices <- length(unique(set_labels$status_group))
choices
# set_labels
# dim(set_labels)

[1] 3

# Data preprocessing

## Missing value imputation




In [16]:
#On some distribution, when importing the dataset, some NA values are replaced by empty strings
#so we set them all tho NA
data1<-replace(dataset,dataset=='',NA)# Replaced to NA

#We then need to see whether there are NA values present in the dataset
#We can do that by removing them using na.omit and compare the dimension of the result with the input
dim(data1)
dim(na.omit(data1))


[1] 59400    40

[1] 27813    40

We can see that the number of rows decreased drastically, so we have to replace those NA values

## Feature selection
We now have to see which features will be kept, and which one introduce redundancy, or whose information aren't relevant to the problem.<br>

///this is just a draft<br>
Looking at the column types, we come accross two types of values : numerical and categorical. For the categorical values, a new encoding is required (like the one hot encoding for example) to obtain numerical values. However, the more unique values for a given columns, the more columns will be added to the dataset, so the features to keep have to take this into account.

In [33]:
head(data1)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
,<int>,<dbl>,<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<chr>,<int>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,69572,6000,2011-03-14,Roman,1390,Roman,34.93809,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
2,8776,0,2013-03-06,Grumeti,1399,GRUMETI,34.69877,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
3,34310,25,2013-02-25,Lottery Club,686,World vision,37.46066,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
4,67743,0,2013-01-28,Unicef,263,UNICEF,38.48616,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
5,19728,0,2011-07-13,Action In A,0,Artisan,31.13085,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
6,9944,20,2011-03-13,Mkinga Distric Coun,0,DWE,39.17280,-4.765587,Tajiri,0,...,per bucket,salty,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe


- The first feature we can look at is the date recorded. According to the description of the features, the row date_recorded corresponds to the date the row was entered. So it is not relevant at all to our probem. We can then remove it from the dataset.
- The next fetaure is num_private. There is no description of it vailable on the datadriven website, so we can try looking at the values it holds : 

In [26]:
#All the values of this column present in this dataset
unique(data1$num_private) 
#Frequency of each value
(table(data1$num_private))

[1]    0   39    5   45    6    3  698   32   15    7   25  102    1   93   14
[16]   34  120   17  213   47    8   41   80  141   20   35  131    4   22   11
[31]   87   61   65  136    2  180   38   62    9   16   23   42   24   12  668
[46]  672   58  150  280  160   50 1776   30   27   10   94   26  450  240  755
[61]   60  111  300   55 1402


    0     1     2     3     4     5     6     7     8     9    10    11    12 
58643    73    23    27    20    46    81    26    46     4     4    11     5 
   14    15    16    17    20    22    23    24    25    26    27    30    32 
    3    35     8    17    14     6     1     5    12     2     2     1    40 
   34    35    38    39    41    42    45    47    50    55    58    60    61 
   10     1     4    30    10     1    36    21     5     1     4     1     3 
   62    65    80    87    93    94   102   111   120   131   136   141   150 
    1    22    15     1    28     1    20     1     7     1     1     1     6 
  160   180   213   240   280   300   450   668   672   698   755  1402  1776 
    1     1     1     1     1     1     1     1     1     1     1     1     1 

We see that more than half the values are zero, which could suggest the value is just missing. Anyway, the values are not diversified enough so we remove that column from our dataset.

In [37]:
#All the values of this column present in this dataset
length(unique(data1$funder)) 
length(unique(data1$installer))
length(unique(data1$wpt_name)) 
length(unique(data1$subvillage)) 

[1] 1898

[1] 2146

- The funder feature is also categorical. Looking at the number of unique values, there are 1898. Encoding this using one hot is not a good idea since it will introduce too many new parameters and will probably have a huge impact on the speed and accuracy of our future models.
- The same observation can be made for the installer, wpt_name and subvillage features.
- We can look at the following features together : region, region_code, lga, district code and ward :

In [51]:
length(unique(data1$lga)) 
length(unique(data1$ward))

#All the values of this column present in this dataset
length(unique(data1$region)) 
#Frequency of each value
(table(data1$region))

#All the values of this column present in this dataset
length(unique(data1$region_code)) 
#Frequency of each value
(table(data1$region_code))

#All the values of this column present in this dataset
length(unique(data1$district_code)) 
#Frequency of each value
(table(data1$district_code))

[1] 125

[1] 2092

[1] 21


       Arusha Dar es Salaam        Dodoma        Iringa        Kagera 
         3350           805          2201          5294          3316 
       Kigoma   Kilimanjaro         Lindi       Manyara          Mara 
         2816          4379          1546          1583          1969 
        Mbeya      Morogoro        Mtwara        Mwanza         Pwani 
         4639          4006          1730          3102          2635 
        Rukwa        Ruvuma     Shinyanga       Singida        Tabora 
         1808          2640          4982          2093          1959 
        Tanga 
         2547 

[1] 27


   1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16 
2201 3024 4379 2513 4040 1609  805  300  390 2640 5300 4639 2093 1979 1808 2816 
  17   18   19   20   21   24   40   60   80   90   99 
5011 3324 3047 1969 1583  326    1 1025 1238  917  423 

[1] 20


    0     1     2     3     4     5     6     7     8    13    23    30    33 
   23 12203 11173  9998  8999  4356  4074  3343  1043   391   293   995   874 
   43    53    60    62    63    67    80 
  505   745    63   109   195     6    12 

- We can right away remove features lga and ward, since they are both categorical and the number of unqiue valeus is very big.
## what conclusion for the 3 others ? 

In [57]:
length(unique(data1$scheme_name)) 

[1] 2697

- For the scheme_name, we see that we have 2967 different categorical values, wo we will drop this feature too.
## why did me remove payment when only 7 values ? 

In [59]:
length(unique(data1$payment)) 
(table(data1$payment))

[1] 7


            never pay                 other          pay annually 
                25348                  1054                  3642 
          pay monthly        pay per bucket pay when scheme fails 
                 8300                  8985                  3914 
              unknown 
                 8157 

In [64]:
length(unique(data1$quantity)) 
(table(data1$quantity))
length(unique(data1$quantity_group)) 
(table(data1$quantity_group))

length(unique(data1$waterpoint_type)) 
(table(data1$waterpoint_type))
length(unique(data1$waterpoint_type_group)) 
(table(data1$waterpoint_type_group))

length(unique(data1$recorded_by)) 
(table(data1$recorded_by))

[1] 5


         dry       enough insufficient     seasonal      unknown 
        6246        33186        15129         4050          789 

[1] 5


         dry       enough insufficient     seasonal      unknown 
        6246        33186        15129         4050          789 

[1] 7


              cattle trough          communal standpipe 
                        116                       28522 
communal standpipe multiple                         dam 
                       6103                           7 
                  hand pump             improved spring 
                      17488                         784 
                      other 
                       6380 

[1] 6


     cattle trough communal standpipe                dam          hand pump 
               116              34625                  7              17488 
   improved spring              other 
               784               6380 

[1] 1


GeoData Consultants Ltd 
                  59400 

- The two features quantity and quantity_group are identical, so we only need to keep one of them. We removed quantity
- For the fetaures waterpoint_type and waterpoint_type_group, we see that the values are similar expect for communal standpipe that is divided into "communal standpipe" and "communal standpipe multiplle" for the waterpoint_type feature. So we chose to keep this feature since it gives a little more variability to the data.
- Lookign at the recorded_by feature, we see that it only has one value which is "GeoData Consultants Ltd", so it doesn't bring any information to the dataset. It has been then removed.

In [65]:
# Delete the columns in cleaning based on the variance, the relevance of the information, etc.
cleaning <- c("date_recorded", "num_private", "funder", "installer", "wpt_name", "subvillage", "lga", "ward", 
              "region_code", "scheme_name", "payment", "quantity", "waterpoint_type_group", "recorded_by")
X<-dataset[,setdiff(colnames(dataset),cleaning)]
X<-replace(X,X=='',NA)#Replaced to NA
X[20:26]

,id,amount_tsh,gps_height,longitude,latitude,basin,region,district_code,population,public_meeting,...,management,management_group,payment_type,water_quality,quality_group,quantity_group,source,source_type,source_class,waterpoint_type
,<int>,<dbl>,<int>,<dbl>,<dbl>,<chr>,<chr>,<int>,<int>,<chr>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,69572,6000,1390,34.93809,-9.856322,Lake Nyasa,Iringa,5,109,True,...,vwc,user-group,annually,soft,good,enough,spring,spring,groundwater,communal standpipe
2,8776,0,1399,34.69877,-2.147466,Lake Victoria,Mara,2,280,NA,...,wug,user-group,never pay,soft,good,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe
3,34310,25,686,37.46066,-3.821329,Pangani,Manyara,4,250,True,...,vwc,user-group,per bucket,soft,good,enough,dam,dam,surface,communal standpipe multiple
4,67743,0,263,38.48616,-11.155298,Ruvuma / Southern Coast,Mtwara,63,58,True,...,vwc,user-group,never pay,soft,good,dry,machine dbh,borehole,groundwater,communal standpipe multiple
5,19728,0,0,31.13085,-1.825359,Lake Victoria,Kagera,1,0,True,...,other,other,never pay,soft,good,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe
6,9944,20,0,39.17280,-4.765587,Pangani,Tanga,8,1,True,...,vwc,user-group,per bucket,salty,salty,enough,other,other,unknown,communal standpipe multiple
7,19816,0,0,33.36241,-3.766365,Internal,Shinyanga,3,0,True,...,vwc,user-group,never pay,soft,good,enough,machine dbh,borehole,groundwater,hand pump
8,54551,0,0,32.62062,-4.226198,Lake Tanganyika,Shinyanga,3,0,True,...,wug,user-group,unknown,milky,milky,enough,shallow well,shallow well,groundwater,hand pump
9,53934,0,0,32.71110,-5.146712,Lake Tanganyika,Tabora,6,0,True,...,vwc,user-group,never pay,salty,salty,seasonal,machine dbh,borehole,groundwater,hand pump


We initially had 40 features. After removing the ones mentionned above, we are left with 26. 

## need to justify those we didnt remove ? 


## Feature engineering



Now that we have already removed some features, we can put the remainig ones in the correct format.<br>
For numerical values, we will replace all the NAs with the median value on that column.<br>
For categorical data, replace them with the most frequent value.

In [67]:
replace_na_with_median_value<-function(vec) {
    median_vec<-median(vec,na.rm=T)
    vec[is.na(vec)]<-median_vec
    vec
}

In [68]:
#distric code 0 ?? a demander
#population 0 -> mean ???
#public_meeting -> plus frequent
#scheme_management -> frequent
#permit -> plus frequent
#construction_year -> int(mean)
#dim(data.frame(X$permit))

#Replacing NA values in categorical features
table(X$permit)
X$permit[is.na(X$permit)] <- tail(names(sort(table(X$permit))), 1)
table(X$permit)

table(X$scheme_management)
X$scheme_management[is.na(X$scheme_management)] <- tail(names(sort(table(X$scheme_management))), 1)
table(X$scheme_management)

table(X$public_meeting)
X$public_meeting[is.na(X$public_meeting)] <- tail(names(sort(table(X$public_meeting))), 1)
table(X$public_meeting)


False  True 
17492 41908 


False  True 
17492 41908 


         Company             None            Other       Parastatal 
            1061                1              766             1680 
Private operator              SWC            Trust              VWC 
            1063               97               72            40670 
 Water authority      Water Board              WUA              WUG 
            3153             2748             2883             5206 


         Company             None            Other       Parastatal 
            1061                1              766             1680 
Private operator              SWC            Trust              VWC 
            1063               97               72            40670 
 Water authority      Water Board              WUA              WUG 
            3153             2748             2883             5206 


False  True 
 5055 54345 


False  True 
 5055 54345 

In [75]:
#For construction_year, some rows have a value 0, before replacing it , we first set 0 to NA so we can use is.na
X$construction_year[X$construction_year==0] <- NA
X$construction_year[is.na(X$construction_year)] <- median(X$construction_year,na.rm=T)
X$construction_year <- as.integer(X$construction_year)

In [77]:
#Same for longitude
X$longitude[X$longitude==0] <- NA
X$longitude[is.na(X$longitude)] <- mean(X$longitude,na.rm=T)

#X$latitude[X$latitude==0] <- NA
#X$latitude[is.na(X$latitude)] <- mean(X$latitude,na.rm=T)
#sum(X$latitude==-0.00000002)

In [78]:
#For permit, the categorical values are bolleans true-false so we can encode them on only one column with 0 and 1, same goes
#for public meeting
X$permit[X$permit=='True'] <- 1
X$permit[X$permit=='False'] <- 0
X$permit <- as.integer(X$permit)

X$public_meeting[X$public_meeting=='True'] <- 1
X$public_meeting[X$public_meeting=='False'] <- 0
X$public_meeting <- as.integer(X$public_meeting)

In [79]:
summary(X)
X[1:20]

       id          amount_tsh         gps_height       longitude    
 Min.   :    0   Min.   :     0.0   Min.   : -90.0   Min.   :29.61  
 1st Qu.:18520   1st Qu.:     0.0   1st Qu.:   0.0   1st Qu.:33.35  
 Median :37062   Median :     0.0   Median : 369.0   Median :35.15  
 Mean   :37115   Mean   :   317.7   Mean   : 668.3   Mean   :35.15  
 3rd Qu.:55657   3rd Qu.:    20.0   3rd Qu.:1319.2   3rd Qu.:37.18  
 Max.   :74247   Max.   :350000.0   Max.   :2770.0   Max.   :40.35  
    latitude          basin              region          district_code  
 Min.   :-11.649   Length:59400       Length:59400       Min.   : 0.00  
 1st Qu.: -8.541   Class :character   Class :character   1st Qu.: 2.00  
 Median : -5.022   Mode  :character   Mode  :character   Median : 3.00  
 Mean   : -5.706                                         Mean   : 5.63  
 3rd Qu.: -3.326                                         3rd Qu.: 5.00  
 Max.   :  0.000                                         Max.   :80.00  
   pop

id,amount_tsh,gps_height,longitude,latitude,basin,region,district_code,population,public_meeting,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment_type,water_quality
<int>,<dbl>,<int>,<dbl>,<dbl>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
69572,6000,1390,34.93809,-9.85632177,Lake Nyasa,Iringa,5,109,1,VWC,0,1999,gravity,gravity,gravity,vwc,user-group,annually,soft
8776,0,1399,34.69877,-2.14746569,Lake Victoria,Mara,2,280,1,Other,1,2010,gravity,gravity,gravity,wug,user-group,never pay,soft
34310,25,686,37.46066,-3.82132853,Pangani,Manyara,4,250,1,VWC,1,2009,gravity,gravity,gravity,vwc,user-group,per bucket,soft
67743,0,263,38.48616,-11.15529772,Ruvuma / Southern Coast,Mtwara,63,58,1,VWC,1,1986,submersible,submersible,submersible,vwc,user-group,never pay,soft
19728,0,0,31.13085,-1.82535885,Lake Victoria,Kagera,1,0,1,VWC,1,1996,gravity,gravity,gravity,other,other,never pay,soft
9944,20,0,39.17280,-4.76558728,Pangani,Tanga,8,1,1,VWC,1,2009,submersible,submersible,submersible,vwc,user-group,per bucket,salty
19816,0,0,33.36241,-3.76636472,Internal,Shinyanga,3,0,1,VWC,1,1996,swn 80,swn 80,handpump,vwc,user-group,never pay,soft
54551,0,0,32.62062,-4.22619802,Lake Tanganyika,Shinyanga,3,0,1,VWC,1,1996,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,milky
53934,0,0,32.71110,-5.14671181,Lake Tanganyika,Tabora,6,0,1,VWC,1,1996,india mark ii,india mark ii,handpump,vwc,user-group,never pay,salty


In [15]:
names(X)

[1] "id"                    "amount_tsh"            "gps_height"           
 [4] "longitude"             "latitude"              "basin"                
 [7] "region"                "district_code"         "population"           
[10] "public_meeting"        "scheme_management"     "permit"               
[13] "construction_year"     "extraction_type"       "extraction_type_group"
[16] "extraction_type_class" "management"            "management_group"     
[19] "payment_type"          "water_quality"         "quality_group"        
[22] "quantity_group"        "source"                "source_type"          
[25] "source_class"          "waterpoint_type"

In [80]:
#one hot conversion
#install.packages('dummies')
library(dummies)

#Now that all fetaures have the correct format, we can encode categorical variables into classes with values 0-1 using
#the one hot encoding
variables_to_keep<-names(X)
X_onehot <- dummy.data.frame(X[,variables_to_keep], sep="_")

dummies-1.5.6 provided by Decision Patterns


Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 

In [17]:
#names(data_factor_onehot)
X_onehot

,id,amount_tsh,gps_height,longitude,latitude,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,...,source_class_groundwater,source_class_surface,source_class_unknown,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other
,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,69572,6000,1390,34.93809,-9.85632177,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
2,8776,0,1399,34.69877,-2.14746569,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,0
3,34310,25,686,37.46066,-3.82132853,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
4,67743,0,263,38.48616,-11.15529772,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
5,19728,0,0,31.13085,-1.82535885,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,0
6,9944,20,0,39.17280,-4.76558728,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
7,19816,0,0,33.36241,-3.76636472,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
8,54551,0,0,32.62062,-4.22619802,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
9,53934,0,0,32.71110,-5.14671181,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0


In [84]:
vars <- c("id", "status_group") ##???
labels_onehot <- dummy.data.frame(set_labels[,vars], sep="_")

#Renaming the encoded features in the labels dataframe
names(labels_onehot)[names(labels_onehot) == "status_group_functional"] <- "functional"
names(labels_onehot)[names(labels_onehot) == "status_group_functional needs repair"] <- "functional_needs_repair"
names(labels_onehot)[names(labels_onehot) == "status_group_non functional"] <- "non_functional"
labels_onehot

#set_labels$status_group[set_labels$status_group=='functional']<-0
#set_labels$status_group[set_labels$status_group=='functional needs repair']<-0.5
#set_labels$status_group[set_labels$status_group=='non functional']<-1

#set_labels$status_group <- as.double(set_labels$status_group)
#set_labels

Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"


,id,functional,functional_needs_repair,non_functional
,<int>,<int>,<int>,<int>
1,69572,1,0,0
2,8776,1,0,0
3,34310,1,0,0
4,67743,0,0,1
5,19728,1,0,0
6,9944,1,0,0
7,19816,0,0,1
8,54551,0,0,1
9,53934,0,0,1


In [86]:
Y<-labels_onehot[,setdiff(colnames(labels_onehot),"id")]
Y<-set_labels[2]
Y[Y == "functional needs repair"] <- "functional_needs_repair"
Y[Y == "non functional"] <- "non_functional"
Y

status_group
<chr>
functional
functional
functional
non_functional
functional
functional
non_functional
non_functional
non_functional


In [87]:
#Removing id from the dataset, since the training dtaa and labels are both ordered in the same way
X_final<-X_onehot[,setdiff(colnames(X_onehot),"id")]
X_final

,amount_tsh,gps_height,longitude,latitude,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,basin_Pangani,...,source_class_groundwater,source_class_surface,source_class_unknown,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other
,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,6000,1390,34.93809,-9.85632177,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
2,0,1399,34.69877,-2.14746569,0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0
3,25,686,37.46066,-3.82132853,0,0,0,0,0,1,...,0,1,0,0,0,1,0,0,0,0
4,0,263,38.48616,-11.15529772,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
5,0,0,31.13085,-1.82535885,0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,0
6,20,0,39.17280,-4.76558728,0,0,0,0,0,1,...,0,0,1,0,0,1,0,0,0,0
7,0,0,33.36241,-3.76636472,1,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
8,0,0,32.62062,-4.22619802,0,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
9,0,0,32.71110,-5.14671181,0,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0


# Model selection

## Model 1

## Model 2

## Model 3

#### Example of simple equation
\begin{equation}
e = mc^2
\end{equation}

#### Example of matrix equation - Cross product formula:

\begin{equation*}
\mathbf{V}_1 \times \mathbf{V}_2 =  \begin{vmatrix}
\mathbf{i} & \mathbf{j} & \mathbf{k} \\
\frac{\partial X}{\partial u} &  \frac{\partial Y}{\partial u} & 0 \\
\frac{\partial X}{\partial v} &  \frac{\partial Y}{\partial v} & 0
\end{vmatrix}
\end{equation*}

#### Example of multiline equation - The Lorenz Equations:

\begin{align}
\dot{x} & = \sigma(y-x) \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = -\beta z + xy
\end{align}

#### Example of Markdown Table:

| This | is   |
|------|------|
|   a  | table|


In [28]:
N<-nrow(X_final)    #Number of examples
n<-ncol(X_final)    #Number of input variables

names(X_final)<-make.names(names(X_final))

In [32]:

DS<-cbind(X_pca,label=Y)
names(DS)<-make.names(names(DS))
DS$status_group<-factor(DS$status_group)

In [33]:
DS

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC151,PC152,PC153,PC154,PC155,PC156,PC157,PC158,PC159,status_group
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,-5694.65598,592.626326,-186.203084,2.3411072,-1.6078650,3.1386680,-2.14335613,0.496976205,1.098876923,-0.3718124,...,8.399314e-16,1.154675e-16,-1.094079e-16,2.982721e-18,-1.266122e-16,-8.708282e-17,-8.376868e-17,-3.070447e-17,1.978124e-17,functional
2,303.63567,743.109148,-21.352283,13.0292677,3.3197694,-3.2369737,1.31350858,1.012054200,-0.650420512,0.8549088,...,-2.835189e-16,3.485710e-16,1.652578e-16,1.478584e-16,-2.378166e-16,-4.537661e-17,-2.264111e-16,8.053468e-17,-1.187746e-17,functional
3,292.07535,34.510625,66.124405,12.2689017,2.6605354,-0.2701010,2.79118688,1.182500162,-0.321827684,0.2028049,...,3.894454e-16,-3.745919e-16,-1.344292e-16,2.071066e-16,-2.207006e-16,5.136620e-18,6.695956e-17,-1.478895e-17,3.631927e-17,functional
4,325.52024,-413.856576,-53.723258,-5.8008588,-57.5539431,0.8034112,-1.13208940,-0.476679276,-0.847686803,-1.1755996,...,-5.965280e-16,-1.441989e-17,-6.388492e-17,-4.721245e-17,5.701019e-17,2.254550e-16,1.916314e-16,-1.778527e-18,-1.442139e-17,non_functional
5,330.59903,-682.621009,-67.889513,-0.7626552,6.5200030,-4.5858135,-1.54205853,2.067721343,-0.564478420,0.2155048,...,-2.835189e-16,7.101524e-17,-1.018896e-16,2.369798e-16,5.405065e-17,1.346823e-16,7.066025e-17,-9.684080e-17,2.411805e-17,functional
6,310.59983,-682.846965,-66.865744,12.8741376,-0.2025347,1.2369008,3.91634214,0.002843854,-0.819028696,-0.9050292,...,9.194034e-16,-1.704474e-15,-1.560997e-16,-3.313731e-17,1.667111e-16,-1.985888e-16,4.638252e-16,-1.446369e-17,-1.141680e-16,functional
7,330.59909,-682.626383,-67.883428,-0.5602538,4.2262372,-2.0422393,-0.57065274,-0.738957002,-0.088323096,-0.8394809,...,4.266336e-16,-1.861575e-16,-5.765076e-17,-1.125463e-17,-5.507472e-17,-5.215161e-17,1.270205e-16,1.997836e-17,1.938762e-17,non_functional
8,330.59909,-682.626880,-67.883551,-0.5672506,4.2629810,-2.1878232,-1.50653340,-1.437341891,0.205704523,0.7416535,...,7.943949e-16,-4.845300e-16,8.692760e-17,1.180229e-16,2.282521e-17,6.575538e-17,3.959317e-17,-1.094754e-16,-7.934275e-18,non_functional
9,330.59927,-682.633044,-67.876729,-0.3394667,1.1867450,-1.4623933,-1.89990028,-0.988586346,-0.286550443,-1.2895820,...,-8.154285e-16,3.238512e-16,-1.428267e-17,-3.648130e-16,1.294565e-16,-6.049996e-17,-3.342533e-16,3.721717e-17,-1.183740e-17,non_functional


In [39]:
data.frame(model$variable.importance)# for rpart


ERROR: Error in data.frame(model$variable.importance): objet 'model' introuvable


In [37]:
#install.packages("rpart.plot")
prp(model)
rpart.plot(model)

ERROR: Error in prp(model): impossible de trouver la fonction "prp"


In [38]:
# plot tree
plot(model, uniform=TRUE,
   main="Classification Tree - Rpart")
text(model, use.n=TRUE, all=TRUE, cex=.8)

ERROR: Error in plot(model, uniform = TRUE, main = "Classification Tree - Rpart"): objet 'model' introuvable


# nnet

In [ ]:
accuracyFromConfusionMatrix <- function(confusion_matrix){
    if(dim(confusion_matrix)[1] < 2){ 
       if(rownames(confusion_matrix) == "nonspam"){
           confusion_matrix <- rbind(confusion_matrix,c(0,0))
           rownames(confusion_matrix)[2] <- "spam"
       }
       if(rownames(confusion_matrix) == "spam"){
           confusion_matrix <- rbind(c(0,0),confusion_matrix)
           rownames(confusion_matrix)[1] <- "nonspam"
       }
    }
    return((confusion_matrix[1,1]+confusion_matrix[2,2])/sum(confusion_matrix))
}


computeResults <- function(Y,Y_hat,explanatory_string="",verbose=TRUE){
    
    confusion_matrix <- table(Y_hat,Y)
    accuracy <- accuracyFromConfusionMatrix(confusion_matrix)
    misclassification_rate <- 1 - accuracy
    
    if(verbose){
        print(paste("[INFO] - Confusion matrix",explanatory_string,":"))
        print(confusion_matrix)
        print(paste("[INFO] - Accuracy",explanatory_string,":",accuracy))
        print(paste("[INFO] - Misclassification rate",explanatory_string,":",misclassification_rate))
    }
    
    return(accuracy)
}

In [ ]:
#test without cross validation

CV_folds <- 10

size_CV <-floor(N/CV_folds)
    
idx_ts<-(((i-1)*size_CV+1):(i*size_CV))  ### idx_ts represents the indices of the test set the i-th fold
X_ts<-X_final[idx_ts,]  
Y_ts<-Y[idx_ts,]
Y_onehot<-labels_onehot[idx_ts,]

idx_tr<-setdiff(1:N,idx_ts) ### idx_tr represents  indices of the training sefor the i-th fold
X_tr<-X_final[idx_tr,]
Y_tr<-Y[idx_tr,]         

DS<-cbind(X_tr,status_group=Y_tr)
DS$status_group<-factor(DS$status_group)
    
    
model<- nnet(status_group~.,data=DS, size=10, maxit=1000, trace=T, MaxNWts=2000)#1000 max iterations ?
Y_hat_ts<- predict(model,X_ts)

In [ ]:
function(x){
   (x-min(x))/(max(x)-min(x))
}

Y_hat <- t(apply(Y_hat_ts, 1, function(x)(x-min(x))/(max(x)-min(x))))
                 
Y_hat[,1] <- ifelse(Y_hat[,1] < 0.99,0,1)
Y_hat[,2] <- ifelse(Y_hat[,2] < 0.99,0,1)
Y_hat[,3] <- ifelse(Y_hat[,3] < 0.99,0,1)
Y_hat
Y_onehot

In [ ]:
Y_onehot<-Y_onehot[,setdiff(colnames(Y_onehot),"id")]
threshold <- 0.5
Y_hat <- ifelse(Y_hat_ts[,1] > threshold,"functional",ifelse(Y_hat_ts[,2] > threshold,"functional_needs_repair","non_functional"))
data.matrix(Y_hat)

In [ ]:
#table(data.frame(Y_hat),data.frame(Y))

require(caret)
#acc <- confusionMatrix(as.factor(Y_hat),as.factor(Y[idx_ts,]))$overall['Accuracy']
confusionMatrix(as.factor(Y_hat),as.factor(Y[idx_ts,]))

In [ ]:
table(Y)

In [ ]:
#change std and error, maybe cross validating and taking the best performing one ?

#nnet model

CV_folds <- 10#10

size_CV <-floor(N/CV_folds)

CV_err<-matrix(0,nrow=1,ncol=CV_folds)


for (i in 1:CV_folds) {
    
    idx_ts<-(((i-1)*size_CV+1):(i*size_CV))  ### idx_ts represents the indices of the test set the i-th fold
    X_ts<-X_pca[idx_ts,]  
    Y_ts<-Y[idx_ts,]
    Y_onehot<-labels_onehot[idx_ts,]
     
    idx_tr<-setdiff(1:N,idx_ts) ### idx_tr represents  indices of the training sefor the i-th fold
    X_tr<-X_pca[idx_tr,]
    Y_tr<-Y[idx_tr,]         
        # Create a dataset including only the first nb_components principal components
    DS<-cbind(X_tr,status_group=Y_tr)
    DS$status_group<-factor(DS$status_group)

    # Model fit (using lm function)
    model<- nnet(status_group~.,data=DS, size=5, decay=1.0e-5, maxit=50, trace=FALSE)#1000 max iterations ?
    #model<- rpart(status_group~.,data=DS)

    # Model predict
    Y_hat_ts<- predict(model,X_ts)
    #Y_hat_ts <- dummy.data.frame(data.frame(Y_hat_ts), sep="_") #for random forest
} 
#print(paste("CV error=",round(apply(CV_err,1,mean),digits=4), " ; std dev=",round(apply(CV_err,1,sd),digits=4)))


#model<- nnet(status_group~.,data=DS, size=3, decay=1.0e-5, maxit=50)
#Y_hat_ts<- predict(model,X_pca)

In [ ]:
Y_hat_ts

# Alternative models





# Conclusions